In [97]:
import requests, time, random
from bs4 import BeautifulSoup
from selenium import webdriver
from six import iteritems
import pandas as pd
from collections import defaultdict

In [2]:
all_profile_info = {}

In [196]:
class Helper:
    @staticmethod
    def printDict(_dict):
        for key, value in iteritems(_dict):
            print(key,':',value)

    @staticmethod
    def printListDic(all_exp):
        for _dict in all_exp:
            printDict(_dict)
            print()
            
    @staticmethod
    def login(browser):
        # browser.get('https://www.linkedin.com/uas/login')
        file = open('config.txt')
        lines = file.readlines()
        username = lines[0]
        password = lines[1]

        elementID = browser.find_element_by_id('username')
        elementID.send_keys(username)
        elementID = browser.find_element_by_id('password')
        elementID.send_keys(password)
        elementID.submit()
        
    @staticmethod
    def get_browser(logged_in=True):
        browser = webdriver.Chrome('driver/chromedriver.exe')
        browser.get('https://www.linkedin.com/uas/login')
        if logged_in:
            Helper.login(browser)
        return browser
    
    @staticmethod
    def scroll(browser, scroll_time = 2):
        SCROLL_PAUSE_TIME = scroll_time
        last_height = browser.execute_script("return document.body.scrollHeight")
        for i in range(3):
            browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(SCROLL_PAUSE_TIME)
            new_height = browser.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
         
    @staticmethod   
    def get_profile_link_and_scroll(browser, url):
        browser.get(url)
        Helper.scroll(browser)
        
    @staticmethod
    def get_profile_id_from_linkedin_url(url):
        url = str(url)
        index = url.find('.com')
        return url[index+4:]
    
    @staticmethod
    def get_profile_url_from_id(_id):
        LINKEDIN_BASE_URL = 'https://www.linkedin.com'
        _id = str(_id)
        
        url = LINKEDIN_BASE_URL + _id
        return url
    
    @staticmethod
    def setBroswerAndGetSoup(browser, profile_url):
        browser.get(profile_url)
        Helper.scroll(browser)
        src = browser.page_source
        soup = BeautifulSoup(src, 'lxml')
        return soup
    
    @staticmethod
    def is_from_IIITA(all_edu_exp):
        IIITA_School_Company = 'https://www.linkedin.com/company/497317/'
        IIITA = 'https://www.linkedin.com/search/results/all/?keywords=IIIT+Allahabad'
        for each_edu_exp in all_edu_exp:
            if each_edu_exp['URL ID'] == IIITA_School_Company:
                return True
            if each_edu_exp['URL ID'] == IIITA:
                return True
        return False
    

In [230]:
# browser = Helper.get_browser(logged_in=False)
browser = Helper.get_browser(logged_in=True)

c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:29: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:21: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead


In [309]:
class Followers:
    LINKEDIN_BASE_URL = 'https://www.linkedin.com'
    def __init__(self, logged_in_browser, scrolls = 1):
        self.browser = logged_in_browser
        self.scrolls = scrolls
        
    def getFollowersList(self):
        followers_url = 'https://www.linkedin.com/feed/followers/'
        browser.get(followers_url)
        time.sleep(2)
        for i in range(100):
            Helper.scroll(self.browser, 2)
        src = browser.page_source
        bs = BeautifulSoup(src, 'lxml')
        lis = []
        all_followers = bs.find('div', {'class': 'feed-following'})
        if all_followers is None:
            return lis
        for each_follower in all_followers.findAll('a'):
            follower_id = each_follower.get('href')
            if follower_id is None:
                continue
            lis.append(follower_id)
        return list(set(list(lis)))
    
    def getFollowersListPage(self):
        followers_url = 'https://www.linkedin.com/company/81519323/admin/analytics/followers/'
        browser.get(followers_url)
        time.sleep(2)
        for i in range(self.scrolls):
            Helper.scroll(self.browser, 2)
        src = browser.page_source
        bs = BeautifulSoup(src, 'lxml')
        lis = []
        all_followers = bs.find('table', {'class': 'org-view-page-followers-module__table table'})
        if all_followers is None:
            return lis
        for each_follower in all_followers.findAll('a'):
            follower_id = each_follower.get('href')
            if follower_id is None:
                continue
            lis.append(follower_id)
        return list(set(list(lis)))

In [313]:
my_followers = Followers(browser,3)
page_foll_lis = my_followers.getFollowersListPage()

In [314]:
print(page_foll_lis)

['/in/abhi4gupta/', '/in/ayushraj0309/', '/in/ayush-kumar-47a9921b0/']


In [305]:
print(len(page_foll_lis))

3


In [306]:
def saveJSON(flw_list, file='followers.json'):
    data = {}
    for pid in flw_list:
        data2 = {}
        data2['Profile URL'] = Helper.get_profile_url_from_id(pid)
        data[pid] = data2
    out_file = open(file, "w")
    json.dump(data, out_file, indent = 6)
    out_file.close()

In [307]:
saveJSON(page_foll_lis, file='page_followers.json')

In [111]:
class InfoScraper:
    def __init__(self, profile_url, logged_in_browser):
        self.profile_id = Helper.get_profile_id_from_linkedin_url(profile_url)
        self.profile_url = profile_url
        self.browser = logged_in_browser
        self.bs = self.setBroswerAndGetSoup()
        self.basicProfileInfo = self.getBasicProfileInfo()
        self.all_experiences = self.getAllExperieceInfo()
        self.all_educations = self.getAllEducationInfo()
        self.is_faculty = Helper.is_from_IIITA(self.all_experiences)
        self.is_student = Helper.is_from_IIITA(self.all_educations)
        self.profile_status = self.connectionStatus()

    def setBroswerAndGetSoup(self):
        self.browser.get(self.profile_url)
        Helper.scroll(self.browser)
        src = self.browser.page_source
        soup = BeautifulSoup(src, 'lxml')
        return soup

    def refreshStatus(self):
        self.bs = self.setBroswerAndGetSoup()
        self.profile_status = self.connectionStatus()
        
    def connectionStatus(self):
        bs = self.bs
        actions_div = bs.find('div', {'class': 'pvs-profile-actions'})
        if actions_div is None:
            return "Error"
        actions = actions_div.findAll('div', {'class': 'entry-point'})
        if ' to connect' in str(actions_div):
            return "Not Connected"
        if 'Pending' in str(actions_div):
            return "Request Pending"
        if actions is None or len(actions) == 0:
            return "Unknown"
        return "Connected"

    def getProfileName(self, bs):
        name_div = bs.find('div', {'class': 'mt2 relative'})
        name_loc = name_div.find_all('div')
        name = name_loc[0].find('h1').get_text().strip()
        return str(name)

    def getProfileLocation(self, bs):
        name_div = bs.find('div', {'class': 'mt2 relative'})
        name_loc = name_div.find_all('div')
        if len(name_loc) == 0:
            return 'None'
        loc = name_loc[-1].find('span').get_text().strip()
        return str(loc)

    def getBasicProfileInfo(self):
        bs = self.bs
        info = {}
        info['Name'] = self.getProfileName(bs)
        info['Location'] = self.getProfileLocation(bs)
        return info
    
    def getExpURLID(self, bs):
        ids = []
        href_classes = bs.findAll('a', {'data-field': 'experience_company_logo'})
        for href_class in href_classes:
            if href_class is None:
                continue
            exp_url = href_class.attrs.get('href', None)
            if exp_url is None:
                continue
            ids.append(exp_url)
        return ids

    def getExpTitle(self, all_exp_spans):
        if len(all_exp_spans) < 2:
            return 'None'
        job_title = all_exp_spans[1].get_text().strip()
        return str(job_title)

    def getExpCompany(self, all_exp_spans):
        if len(all_exp_spans) < 4:
            return 'None'
        double_text = str(all_exp_spans[3].get_text().strip())
        strip_index = len(double_text)//2
        company = double_text[0:strip_index].split(' · ')[0]
        return str(company)

    def getExpTimePeriod(self, all_exp_spans):
        if len(all_exp_spans) < 7:
            return 'None'
        time_period = all_exp_spans[6].get_text().strip().split(' · ')[0]
        return str(time_period)

    def getExpInfo(self, exp):
        info = {}
        all_exp_spans = exp.find_all('span')
        job_title = self.getExpTitle(all_exp_spans)
        company = self.getExpCompany(all_exp_spans)
        time_period = self.getExpTimePeriod(all_exp_spans)
        
        info['Job Title'] = job_title
        info['Company'] = company
        info['Time Period'] = time_period
        return info
        
    def getExperienceSetions(self, bs):
        profile_sections = bs.findAll('section')
        exp_profile_section = None
        for profile_section in profile_sections:
            x = profile_section.find('div', {'id': 'experience'})
            if x is not None:
                exp_profile_section = profile_section
                break
        if exp_profile_section is None:
            return {}
        exp_sections = exp_profile_section.find_all('div', {'class': 'display-flex flex-column full-width align-self-center'})
        return exp_sections

    def getAllExperieceInfo(self):
        bs = self.bs
        all_experiences = []
        company_ids = self.getExpURLID(bs)
        exp_sections = self.getExperienceSetions(bs)
        for exp in exp_sections:
            all_experiences.append(self.getExpInfo(exp))
        for i in range(len(all_experiences)):
            if i >= len(company_ids):
                all_experiences[i]['URL ID'] = 'None'
                continue
            all_experiences[i]['URL ID'] = str(company_ids[i])
        return all_experiences

    def getEduSchoolID(self, edu):
        href_class = edu.findAll('a')
        if href_class is None or len(href_class) == 0:
            return 'None'
        school_url = href_class[0].attrs.get('href', None)
        if school_url is None:
            return 'None'
        return str(school_url)

    def getEduSchool(self, all_edu_spans):
        if len(all_edu_spans) < 2:
            return 'None'
        job_title = all_edu_spans[1].get_text().strip()
        return str(job_title)

    def getEduCourse(self, all_edu_spans):
        if len(all_edu_spans) < 4:
            return 'None'
        course_and_discipline = all_edu_spans[4].get_text().strip().split(',')
        if len(course_and_discipline) < 1:
            return 'None'
        course = course_and_discipline[0]
        return str(course)

    def getEduDiscipline(self, all_edu_spans):
        if len(all_edu_spans) < 4:
            return 'None'
        line_text = all_edu_spans[4].get_text()
        course_and_discipline = line_text.strip().split(',')
        if len(course_and_discipline) < 2:
            return 'None'
        ind = line_text.find(', ')
        discipline = line_text[ind+1:]
        return str(discipline)

    def getEduTimePeriod(self, all_edu_spans):
        if len(all_edu_spans) < 8:
            return 'None'
        time_period = all_edu_spans[7].get_text().strip()
        return str(time_period)

    def getEduInfo(self, edu):
        info = {}
        # print(edu)
        all_edu_spans = edu.find_all('span')
        school_url_id = self.getEduSchoolID(edu)
        school = self.getEduSchool(all_edu_spans)
        course = self.getEduCourse(all_edu_spans)
        discipline = self.getEduDiscipline(all_edu_spans)
        time_period = self.getEduTimePeriod(all_edu_spans)
        info['URL ID'] = school_url_id
        info['Insititution'] = school
        info['Course'] = course
        info['Discipline'] = discipline
        info['Time Period'] = time_period
        return info
        
    def getEducationSetions(self, bs):
        profile_sections = bs.findAll('section')
        edu_profile_section = None
        for profile_section in profile_sections:
            x = profile_section.find('div', {'id': 'education'})
            if x is not None:
                edu_profile_section = profile_section
                break
        if edu_profile_section is None:
            return {}
        edu_sections = edu_profile_section.find_all('div', {'class': 'display-flex flex-column full-width align-self-center'})
        return edu_sections

    def getAllEducationInfo(self):
        bs = self.bs
        all_educations = []
        edu_sections = self.getEducationSetions(bs)
        for edu in edu_sections:
            all_educations.append(self.getEduInfo(edu))
        return all_educations

In [127]:
# profile_info = InfoScraper('https://www.linkedin.com/in/abhi4gupta/', browser)
# profile_info = InfoScraper('https://www.linkedin.com/in/dr-sunny-sharma-64887918/', browser)
purl = 'https://www.linkedin.com/in/anupam-agrawal-a822b011/'
profile_info_2 = InfoScraper(purl, browser)
all_profile_info[Helper.get_profile_id_from_linkedin_url(purl)] = profile_info_2

In [128]:
print(all_profile_info['/in/anupam-agrawal-a822b011/'].is_faculty)

True


In [129]:
print(profile_info_2.all_educations)

[{'URL ID': 'https://www.linkedin.com/search/results/all/?keywords=University+of+Bedfordshire%2C+United+Kingdom', 'Insititution': 'University of Bedfordshire, United Kingdom', 'Course': 'Post-Doctoral Research Fellow', 'Discipline': ' Computer Science', 'Time Period': '2007 - 2009'}, {'URL ID': 'https://www.linkedin.com/company/497317/', 'Insititution': 'Indian Institute Of Information Technology', 'Course': 'PhD', 'Discipline': ' Computer Science', 'Time Period': '2001 - 2006'}, {'URL ID': 'https://www.linkedin.com/search/results/all/?keywords=Indian+Institute+of+Technology+Madras', 'Insititution': 'Indian Institute of Technology Madras', 'Course': 'Master of Technology - MTech', 'Discipline': ' Computer Science', 'Time Period': '1993 - 1995'}]


In [152]:
class MyProfile:
    LINKEDIN_BASE_URL = 'https://www.linkedin.com'
    def __init__(self, logged_in_browser):
        self.browser = logged_in_browser
        self.followers = self.getFollowersList()
        self.iiit_student, self.iiit_faculty = self.getIIITList(self.followers)
        
    def getFollowersList(self):
        followers_url = 'https://www.linkedin.com/feed/followers/'
        browser.get(followers_url)
        time.sleep(2)
        src = browser.page_source
        bs = BeautifulSoup(src, 'lxml')
        lis = []
        all_followers = bs.find('div', {'class': 'feed-following'})
        if all_followers is None:
            return lis
        for each_follower in all_followers.findAll('a'):
            follower_id = each_follower.get('href')
            if follower_id is None:
                continue
            lis.append(follower_id)
            profile_info = None
            if follower_id not in all_profile_info:
                profile_info = InfoScraper('https://www.linkedin.com'+follower_id, browser)
            else:
                profile_info = all_profile_info[follower_id]
            all_profile_info[profile_info.profile_id] = profile_info
        return list(set(list(lis)))
    
    def getIIITList(self, users):
        stud = []
        fac = []
        for user in users:
            print(user)
            if user in all_profile_info:
                if all_profile_info[user].is_student:
                    stud.append(user)
                if all_profile_info[user].is_faculty:
                    fac.append(user)
            else:
                print('Not in dict')
        return stud, fac
    
    def send_connection_request_with_text(self, profile_url, text):
        self.browser.get(profile_url)
        time.sleep(1)
        src = self.browser.page_source
        bs = BeautifulSoup(src, 'lxml')
        
        actions_divs = bs.findAll('div', {'class': 'pvs-profile-actions'})
        if len(actions_divs) == 0:
            return
        ids = []
        for actions_div in actions_divs:
            buttons = actions_div.findAll('button')
            if len(buttons) == 0:
                continue
            _id = None
            for button in buttons:
                cnnt = button.get('aria-label')
                if cnnt is None:
                    continue
                if ' to connect' in str(cnnt):
                    _id = button.get('id')
                    break
            if _id is None:
                continue
            ids.append(_id)
        if len(ids) == 0:
            return
        self.browser.find_element_by_id(ids[-1]).click()
        time.sleep(1)
        src = self.browser.page_source
        bs = BeautifulSoup(src, 'lxml')
        _id = None
        for x in bs.find_all('button'):
            if x.get('aria-label') == 'Add a note':
                _id = x.get('id')
                break
        if _id is None:
            return
        self.browser.find_element_by_id(_id).click()
        time.sleep(1)
        INV_REQUEST = text
        elementID = browser.find_element_by_id('custom-message')
        elementID.send_keys(INV_REQUEST)
        
        src = self.browser.page_source
        bs = BeautifulSoup(src, 'lxml')
        _id = None
        for x in bs.find_all('button'):
            if x.get('aria-label') == 'Send now':
                _id = x.get('id')
                break
        if _id is None:
            return
        self.browser.find_element_by_id(_id).click()
   
    def send_text_to_connection(self, profile_url, text):
        self.browser.get(profile_url)
        # time.sleep(1)
        src = self.browser.page_source
        bs = BeautifulSoup(src, 'lxml')
        x = bs.find('a', {'class': 'message-anywhere-button pvs-profile-actions__action artdeco-button'})
        if x is None:
            return
        link = x.get('href')
        if link is None:
            return
        full_link = 'https://www.linkedin.com' + link
        self.browser.get(full_link)
        time.sleep(1)

        src = self.browser.page_source
        bs = BeautifulSoup(src, 'lxml')

        message_text = self.browser.find_element_by_css_selector('div.msg-form__contenteditable > p')
        message_text.send_keys(text)
        message_text.submit()
        
        time.sleep(3)

    
    def sendAnnouncement(self, users, inbox_text, connection_req_text):
        for user in users:
            profile_url = Helper.get_profile_url_from_id(user)
            profile_info = None
            if user not in all_profile_info:
                profile_info = InfoScraper(profile_url, self.browser)
                all_profile_info[user] = profile_info
            else:
                profile_info = all_profile_info[user]
                profile_info.refreshStatus() # Refresh Status
            if profile_info is None:
                return
            status = profile_info.profile_status
            print(status)
            if status == 'Connected':
                self.send_text_to_connection(profile_url, inbox_text)
            elif status == 'Not Connected':
                self.send_connection_request_with_text(profile_url, connection_req_text)
        return

In [153]:
my_profile = MyProfile(browser)

/in/soumabrata-bhattacharya-9b190a1a9/
/in/shubhankar-raj-457b30231/
/in/shivam-gupta-52302516b/
/in/abhi4gupta/
/in/aniket-mallick-75572416b/
/in/sankarsan-bhattacheryya-003526173/
/in/harshit-narula-2a82ba162/
/in/richik-bhattacharjee/
/in/raunak-rathour-84a7b71aa/
/in/harsh-arora-3a8a0a1a8/
/in/utkarsh--priyam/
/in/sougata-das-4132a616b/
/in/utkarsh-pandey-95015b170/
/in/shubham-kumar-7a75851a2/
/in/mitali-seth-4649801a1/
/in/isha-goel30/
/in/sambuddha-koner-98732b206/
/in/siddhi-singh-8653801b1/
/in/prateek-m-058795114/
/in/kapil-tomar-a2b31118b/


In [154]:
lis = ['/in/utkarsh--priyam/']
my_profile.sendAnnouncement(lis, 'Text Sent!', 'Hi! Please accept my request')

Connected


In [121]:
lis = my_profile.followers
iiit_stu, iiit_fac = my_profile.getIIITList(lis)

In [122]:
print(iiit_stu)
print(iiit_fac)

[]
[]


In [35]:
test_profile_urls = [
    'https://www.linkedin.com/in/anupam-agrawal-a822b011/',
    'https://www.linkedin.com/in/shivam-gupta-52302516b/',
    'https://www.linkedin.com/in/isha-goel30/',
    'https://www.linkedin.com/in/richik-bhattacharjee/',
    'https://www.linkedin.com/in/sougata-das-4132a616b/',
    'https://www.linkedin.com/in/abhi4gupta/',
    'https://www.linkedin.com/in/dr-sunny-sharma-64887918/',
    'https://www.linkedin.com/in/sankarsan-bhattacheryya-003526173/',
    'https://www.linkedin.com/in/soumabrata-bhattacharya-9b190a1a9/',
    'https://www.linkedin.com/in/arun-gangwar-14a428202/',
    'https://www.linkedin.com/in/avijit-agarwal-06b495148/',
    'https://www.linkedin.com/in/sanyam-garg-22a487135/',
    'https://www.linkedin.com/in/prateek-m-058795114/',
    'https://www.linkedin.com/in/anish-goswami-49a3b7185/',
    'https://www.linkedin.com/in/pranal-maheshwari-31a850192/',
    'https://www.linkedin.com/in/abhigyansharma29/',
    'https://www.linkedin.com/in/blackdchase/',
    'https://www.linkedin.com/in/raunak-rathour-84a7b71aa/',
    'https://www.linkedin.com/in/utkarsh--priyam/',
]

In [36]:
test_followers = [Helper.get_profile_id_from_linkedin_url(each_profile) for each_profile in test_profile_urls ]
print(test_followers)

['/in/anupam-agrawal-a822b011/', '/in/shivam-gupta-52302516b/', '/in/isha-goel30/', '/in/richik-bhattacharjee/', '/in/sougata-das-4132a616b/', '/in/abhi4gupta/', '/in/dr-sunny-sharma-64887918/', '/in/sankarsan-bhattacheryya-003526173/', '/in/soumabrata-bhattacharya-9b190a1a9/', '/in/arun-gangwar-14a428202/', '/in/avijit-agarwal-06b495148/', '/in/sanyam-garg-22a487135/', '/in/prateek-m-058795114/', '/in/anish-goswami-49a3b7185/', '/in/pranal-maheshwari-31a850192/', '/in/abhigyansharma29/', '/in/blackdchase/', '/in/raunak-rathour-84a7b71aa/', '/in/utkarsh--priyam/']


In [24]:
# Run InfoScapper on list
for profile_url in test_profile_urls:
    profile_id = Helper.get_profile_id_from_linkedin_url(profile_url)
    profile_info = None
    if profile_id not in all_profile_info:
        profile_info = InfoScraper(profile_url, browser)
        all_profile_info[profile_id] = profile_info
    else:
        profile_info = all_profile_info[profile_id]
    print(profile_id, '\t, Student :', profile_info.is_student, '\t, Faculty :', profile_info.is_faculty)

/in/anupam-agrawal-a822b011/ 	, Student : True 	, Faculty : True
/in/shivam-gupta-52302516b/ 	, Student : False 	, Faculty : False
/in/isha-goel30/ 	, Student : False 	, Faculty : False
/in/richik-bhattacharjee/ 	, Student : False 	, Faculty : False
/in/sougata-das-4132a616b/ 	, Student : False 	, Faculty : False
/in/abhi4gupta/ 	, Student : True 	, Faculty : False
/in/dr-sunny-sharma-64887918/ 	, Student : False 	, Faculty : True
/in/sankarsan-bhattacheryya-003526173/ 	, Student : False 	, Faculty : False
/in/soumabrata-bhattacharya-9b190a1a9/ 	, Student : False 	, Faculty : False
/in/arun-gangwar-14a428202/ 	, Student : False 	, Faculty : False
/in/avijit-agarwal-06b495148/ 	, Student : False 	, Faculty : False
/in/sanyam-garg-22a487135/ 	, Student : False 	, Faculty : False
/in/prateek-m-058795114/ 	, Student : True 	, Faculty : False
/in/anish-goswami-49a3b7185/ 	, Student : True 	, Faculty : False
/in/pranal-maheshwari-31a850192/ 	, Student : True 	, Faculty : False
/in/abhigyansh

In [37]:
class OtherProfileInfo:
    def __init__(self, logged_in_browser):
        self.browser = logged_in_browser
        
    def getPeopleAlsoViewedSection(self, bs):
        profile_sections = bs.findAll('section')
        req_profile_section = None
        for profile_section in profile_sections:
            if 'People also viewed' in str(profile_section):
                return profile_section
        return None

    def getNewProfileIDs(self, profile, profilesQueued):
        profilesID = []
        bs = self.setBroswerAndGetSoup(profile)
        people_also_viewed_section = self.getPeopleAlsoViewedSection(bs)
        if people_also_viewed_section is None:
            return None
        all_links = people_also_viewed_section.findAll('a')
        for link in all_links:
            userID = link.get('href')
            if((userID is not None) and (userID not in profilesQueued) and (userID not in visitedProfiles)):
                profilesID.append(userID)
        return profilesID
    
    def setBroswerAndGetSoup(self, profile):
        self.browser.get(profile)
        src = self.browser.page_source
        soup = BeautifulSoup(src, 'lxml')
        return soup


In [38]:
other_profile = OtherProfileInfo(browser)
visitedProfiles = []
profilesQueued = []
other_profile.getNewProfileIDs('https://www.linkedin.com/in/isha-goel30/', profilesQueued)

['/in/abhi4gupta/',
 '/in/dakshita/',
 '/in/prachibindal/',
 '/in/naina-gupta-99a64317a/',
 '/in/muskan-maheshwari-1659aa190/']

In [92]:
import json

In [164]:
global_profile_info = {}

In [161]:
class Exporter(object):
    def __init__(self):
        self.all_ids = []
        self.csv_data = {}
        
#     def JSONimporter(self):
        
        
    def JSONexporter(self, data = None, file='out.json'):
        if data is None:
            data = {}
            lis = list(all_profile_info.keys())
            for pid in lis:
                data2 = {}
                data2_info = {}
                data2_edu = []
                data2_exp = []
                pobj = all_profile_info[pid]
                bas_info = pobj.basicProfileInfo
                data2_info['Name'] = bas_info['Name']
                data2_info['Location'] = bas_info['Location']
                    
                for edu in pobj.all_educations:
                    each_data2_edu = {}
                    each_data2_edu['URL ID'] = edu['URL ID']
                    each_data2_edu['Insititution'] = edu['Insititution']
                    each_data2_edu['Course'] = edu['Course']
                    each_data2_edu['Discipline'] = edu['Discipline']
                    each_data2_edu['Time Period'] = edu['Time Period']
                    data2_edu.append(each_data2_edu)
                    
                for exp in pobj.all_experiences:
                    each_data2_exp = {}
                    each_data2_exp['Job Title'] = exp['Job Title']
                    each_data2_exp['Company'] = exp['Company']
                    each_data2_exp['Time Period'] = exp['Time Period']
                    data2_exp.append(each_data2_edu)
                    
                data2['Basic Info'] = data2_info
                data2['Education'] = data2_edu
                data2['Experience'] = data2_exp
                data2['Is Student'] = pobj.is_student
                data2['Is Faculty'] = pobj.is_faculty
                data2['Connection Status'] = pobj.profile_status
                data[pid] = data2
        out_file = open(file, "w")
        json.dump(data, out_file, indent = 6)
        out_file.close()
    
    def fillData(self, lis):
        if lis is None:
            return {}
        csv_data = {}
        _profiles = []
        _courses = []
        _discipline = []
        _time = []
        for pid in lis:
            pobj = all_profile_info[pid]
            for edu in pobj.all_educations:
                _profiles.append(Helper.get_profile_id_from_linkedin_url(pid))
                _courses.append(edu['Course'])
                _discipline.append(edu['Discipline'])
                _time.append(edu['Time Period'])
        csv_data['ProfileID'] = _profiles
        csv_data['Course'] = _courses
        csv_data['Discipline'] = _discipline
        csv_data['Time Period'] = _time
        return csv_data
    
    def saveXLSX(self, lis=None, name = 'out.xlsx'):
        csv_data = self.fillData(lis)
        if lis is None:
            self.all_ids = list(all_profile_info.keys())
            self.csv_data = self.fillData(self.all_ids)
            csv_data = self.csv_data
        df = pd.DataFrame(csv_data)
        df.to_excel(name)

In [162]:
exporter = Exporter()
# exporter.saveXLSX(lis=test_followers, name='test2.xlsx')
exporter.JSONexporter()

In [160]:
print(list(all_profile_info.keys()))

['/in/anupam-agrawal-a822b011/', '/in/harsh-arora-3a8a0a1a8/', '/in/sambuddha-koner-98732b206/', '/in/kapil-tomar-a2b31118b/', '/in/mitali-seth-4649801a1/', '/in/utkarsh--priyam/', '/in/sankarsan-bhattacheryya-003526173/', '/in/prateek-m-058795114/', '/in/raunak-rathour-84a7b71aa/', '/in/harshit-narula-2a82ba162/', '/in/shubham-kumar-7a75851a2/', '/in/siddhi-singh-8653801b1/', '/in/isha-goel30/', '/in/abhi4gupta/', '/in/shivam-gupta-52302516b/', '/in/richik-bhattacharjee/', '/in/sougata-das-4132a616b/', '/in/utkarsh-pandey-95015b170/', '/in/aniket-mallick-75572416b/', '/in/abhirup-das-740981178/', '/in/subharoop-ghosh-943669150/', '/in/dr-sunny-sharma-64887918/', '/in/soumabrata-bhattacharya-9b190a1a9/', '/in/arun-gangwar-14a428202/', '/in/avijit-agarwal-06b495148/', '/in/sanyam-garg-22a487135/', '/in/anish-goswami-49a3b7185/', '/in/pranal-maheshwari-31a850192/', '/in/abhigyansharma29/', '/in/blackdchase/', '/in/shubhankar-raj-457b30231/', '/in/puja-kumari-948732181/']


In [86]:
class ProfilesByFilter(object):
    def __init__(self, logged_in_browser):
        self.browser = logged_in_browser
    
    def getFilteredFaculty(self):
        ids = []
        for prof_id, prof_obj in iteritems(all_profile_info):
            if prof_obj.is_faculty is True:
                ids.append(prof_id)
        return ids
    
    # def getFilteredCurrentStudent(self):
    #     ids = []
    #     for prof_id, prof_obj in iteritems(all_profile_info):
    #         for edu in prof_obj.all_educations:
    #             if edu['Time Period'] and ('bachelor' in edu['Course'].lower() or 'btech' in  edu['Course'].lower()):
    #                 ids.append(prof_id)
    #     return ids
    
    def getFilteredBranch(self, course):
        ids = []
        for prof_id, prof_obj in iteritems(all_profile_info):
            if course == 'ug':
                for edu in prof_obj.all_educations:
                    if edu['Course'] and ('bachelor' in edu['Course'].lower() or 'btech' in  edu['Course'].lower()):
                        ids.append(prof_id)
            if course == 'pg':
                for edu in prof_obj.all_educations:
                    if edu['Course'] and ('master' in edu['Course'].lower() or 'mtech' in  edu['Course'].lower() or 'mba' in  edu['Course'].lower()):
                        ids.append(prof_id)
        return ids
    
    def getFilteredStudent(self):
        ids = []
        for prof_id, prof_obj in iteritems(all_profile_info):
            if prof_obj.is_student is True:
                ids.append(prof_id)
        return ids
    
    def getFiltered(self, course = None, student = False, faculty = False):
        if faculty is True:
            return self.getFilteredFaculty()
        if student is True:
            return self.getFilteredStudent()
        if course is not None:
            return self.getFilteredBranch(course)
        return []
    

In [130]:
filtered_profiles = ProfilesByFilter(browser)
print(filtered_profiles.getFiltered(student = True))

['/in/anupam-agrawal-a822b011/', '/in/utkarsh--priyam/', '/in/prateek-m-058795114/', '/in/raunak-rathour-84a7b71aa/', '/in/abhi4gupta/', '/in/anish-goswami-49a3b7185/', '/in/pranal-maheshwari-31a850192/', '/in/abhigyansharma29/', '/in/blackdchase/']


In [88]:
ug_stu = filtered_profiles.getFiltered(course = 'ug')
print(ug_stu)

['/in/harsh-arora-3a8a0a1a8/', '/in/sambuddha-koner-98732b206/', '/in/kapil-tomar-a2b31118b/', '/in/mitali-seth-4649801a1/', '/in/utkarsh--priyam/', '/in/sankarsan-bhattacheryya-003526173/', '/in/prateek-m-058795114/', '/in/raunak-rathour-84a7b71aa/', '/in/harshit-narula-2a82ba162/', '/in/shubham-kumar-7a75851a2/', '/in/siddhi-singh-8653801b1/', '/in/isha-goel30/', '/in/abhi4gupta/', '/in/richik-bhattacharjee/', '/in/sougata-das-4132a616b/', '/in/utkarsh-pandey-95015b170/', '/in/aniket-mallick-75572416b/', '/in/abhirup-das-740981178/', '/in/subharoop-ghosh-943669150/', '/in/soumabrata-bhattacharya-9b190a1a9/', '/in/arun-gangwar-14a428202/', '/in/sanyam-garg-22a487135/', '/in/anish-goswami-49a3b7185/', '/in/pranal-maheshwari-31a850192/', '/in/abhigyansharma29/', '/in/blackdchase/', '/in/shubhankar-raj-457b30231/']


In [131]:
exporter.saveXLSX(lis=ug_stu, name='ug_students.xlsx')

In [81]:
import ipywidgets as widgets
from IPython.display import display

In [82]:
v = 'Hell'
ALL = 'ALL'

import pandas as pd
import numpy as np
df = pd.read_csv('test_linkedin.csv')

In [83]:
def unique_sorted_values_plus_ALL(array):
    unique = array.unique().tolist()
    unique.sort()
    unique.insert(0, ALL)
    return unique

In [62]:
df

,Name,Year,Course,PID
0,Abhishek,2018,Bachelor of Technology - Btech,/in/abhi4gupta/
1,Ayush,2019,B.tech,/in/ayushraj0309/


In [65]:
from ipywidgets import widgets 
lbl1=widgets.Label('Course') 
display(lbl1) 
text1=widgets.Text() 
display(text1) 
lbl2=widgets.Label('Year') 
display(lbl2) 
text2=widgets.Text() 
display(text2) 
btn=widgets.Button(description="Get filtered csv") 
display(btn) 
lbl3=widgets.Label()
# display(df)
display(lbl3)
def filter(b):
    _course = text1.value
    _year = text2.value
    print('l1'+text1.value+text2.value)
    print(_course+_year)
    x=text1.value
    y=text2.value
    # df = df[df.Year == y]
    # display(df)
    df2 = df
    if  x!= '':
        df2 = df[df.Year == int(y)]
    if y != '':
        df2 = df[df.Year == int(y)]
    # display(df2)
    # df = df[df.Year == int(y)]
    pid_str =  ''
    for pid in df2.PID:
        pid_str += str(pid)
    lbl3.value = pid_str
    # display(df[df.Year == _year])
    # lbl3.value="result= "+str(x+y)
    # lbl3.value=str(x+y)
btn.on_click(filter)

Label(value='Course')

Text(value='')

Label(value='Year')

Text(value='')

Button(description='Get filtered csv', style=ButtonStyle())

Label(value='')

In [137]:
lis = ['/in/puja-kumari-948732181/']
my_profile.sendAnnouncement(lis, 'Text Sent!', 'Fuck utkarsh nigga!(obviously by utkarsh. Choti bacchi ho kya)')

Not Connected


NameError: name 'ids' is not defined